In [1]:
import pandas as pd
import keras
import numpy as np
import pickle as pk
from io import StringIO
from scipy import spatial
from sklearn.feature_selection import chi2
from keras.models import Model
from keras.layers import LSTM,Dense,Input,Bidirectional, CuDNNLSTM , Dropout
from nltk.tokenize.treebank import TreebankWordTokenizer
from scipy import spatial
from random import shuffle
import pickle as pk
from keras.callbacks import ModelCheckpoint
import gc
import gzip
import gensim 
import logging
 


Using TensorFlow backend.


In [2]:
data_url = "/Users/mohammad/Documents/Internship-IAI/Data-days-stuff/divar_posts_dataset.csv"
data = pd.read_csv(data_url)

In [61]:
data_desc_title = data['desc'] + ' ' + data['title']
data_desc_title.shape

(947635,)

In [62]:
data_desc_title = data_desc_title.str.replace('.', ' ')
data_desc_title = data_desc_title.str.replace('?', ' ')
data_desc_title = data_desc_title.str.replace('!', ' ')
data_desc_title = data_desc_title.str.replace('،', ' ')
data_desc_title = data_desc_title.str.replace('\n', ' ')
data_desc_title = data_desc_title.str.replace(',', ' ')
data_desc_title = data_desc_title.str.replace('؟', ' ')
data_desc_title = data_desc_title.str.replace('!', ' ')

In [ ]:
documents = data_desc_title.str.split()

In [ ]:
# logging.basicConfig(format=’%(asctime)s : %(levelname)s : %(message)s’, level=logging.INFO)
model2 = gensim.models.Word2Vec(
    documents,
    size=60,
    window=10,
    min_count=7,
    workers=10)

In [11]:
model2.train(documents, total_examples=len(documents), epochs=10)

(147048031, 178045330)

In [81]:
model2.build_vocab(['زندگی'], update=True)

In [82]:
model2.wv.add(['<UNK>'], [(np.ones(60))],replace = True)
model2.wv.add(['<PAD>'], [(np.zeros(60))],replace = True)
model2.wv['<UNK>']

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [83]:
model2.save('divar_embeddings')

In [3]:
# model2 = gensim.models.Word2Vec.load('divar_embeddings')

In [13]:
descs = data['desc']

In [14]:
descs = descs.str.replace('.', ' . ')
descs = descs.str.replace('?', ' ؟ ')
descs = descs.str.replace('!', ' ! ')
descs = descs.str.replace('،', ' ، ')
# descs = descs.str.replace('\n', ' . ')
descs = descs.str.replace(',', ' ، ')
descs = descs.str.replace('؟ ', ' ؟ ')
# descs = descs.str.replace('...', ' ... ')
descs = descs.str.replace(':', ' : ')
descs = descs.str.replace('_', ' _ ')
descs = descs.str.replace('-', ' - ')

In [15]:
descs = descs.str.split()

In [16]:
puncs = ['!', '.', '?', ')', '(', '،', '؟', ',', ';', '-', '_', '«', ':', '...', '»', '/']
X_train = []
Y_train = []
window = 10
pads = window *['<PAD>']
for sent in descs[:500000]:
    intersect = set(sent).intersection(set(puncs))
    if len(intersect) == 0:
        continue;
    sent2 = pads + sent + pads
    for w in sent:
        ytmp = (len(puncs) + 1) * [0]
        if w in intersect:
            ytmp[puncs.index(w)] = 1
        else:
            ytmp[len(puncs)] = 1
        xtmp = sent2[sent2.index(w) - window:sent2.index(w)]
        X_train.append(xtmp)
        Y_train.append(ytmp)

In [17]:
MAX_SEQ = 10
def wordToVec(data):
    for s in range(len(data)):
        n = MAX_SEQ - len(data[s])
        if n < 0:
            data[s] = data[s][:MAX_SEQ]
        else:
            for i in range(n):
                data[s].append('<PAD>')
        for v in range(len(data[s])):
            if data[s][v] not in model2.wv:
                data[s][v] = model2.wv['<UNK>']
            else:
                data[s][v] = model2.wv[data[s][v]]
    return np.array(data)

In [18]:
X_validation = X_train[450000:500000].copy()
Y_validation = Y_train[450000:500000].copy()
X_train = X_train[0:200000].copy()
Y_train = Y_train[0:200000].copy()

In [19]:
X_train = wordToVec(X_train)
X_validation = wordToVec(X_validation)

In [35]:
Y_train = np.array(Y_train)
Y_validation = np.array(Y_validation)

In [32]:
BATCH_SIZE = 512
input_layer = Input(batch_shape = (None, MAX_SEQ, 60))
lstm_layer1 = LSTM(units=100, return_sequences=True)(input_layer)
lstm_layer2 = LSTM(units=100, dropout=0.25)(lstm_layer1)

lstm_out1 = Dense(50, activation="relu" )(lstm_layer2)

output_layer = Dense(17, activation="softmax")(lstm_out1)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy',
              optimizer='adam' , metrics=['accuracy'] )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10, 60)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 100)           64400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 17)                867       
Total params: 150,717
Trainable params: 150,717
Non-trainable params: 0
_________________________________________________________________


In [37]:
EPOCH_SIZE = 1
model.fit(X_train, Y_train,validation_data=(X_validation,Y_validation), epochs=EPOCH_SIZE, batch_size=BATCH_SIZE  , verbose=1)

Train on 200000 samples, validate on 50000 samples
Epoch 1/1
200000/200000 [==============================] - 63s 317us/step - loss: 0.3483 - acc: 0.8868 - val_loss: 0.4486 - val_acc: 0.8651
